In [1]:
from pathflowai.utils import load_sql_df, df2sql
import numpy as np

nonechucks may not work properly with this version of PyTorch (1.4.0). It has only been tested on PyTorch versions 1.0, 1.1, and 1.2


In [2]:
patch_size=256#128

In [3]:
df=load_sql_df("bcc/patch_info_new.db",patch_size)

In [4]:
annots=df.columns[6:].values
df['Benign']=0.
df[[col for col in annots if col!="Cancer"]]=0.
df['Benign']=(df['Cancer']==0.).astype(float)
df['annotation']=annots[np.argmax(df.iloc[:,6:].values,1).flatten()]
df.iloc[:,6:].sum(0)

Benign    984206.00000
Cancer    126399.45308
dtype: float64

In [5]:
df2sql(df, "bcc/patch_info_update.db", patch_size, mode='replace')

In [8]:
# df['ID'].unique()


In [9]:
df2=load_sql_df("complete/patch_info.db",patch_size)

In [10]:
annots=df2.columns[6:].values

In [11]:
df2['Complete_Area']=(df2['Incomplete_Area']==0.).astype(float)

In [12]:
df2['annotation']=np.vectorize(lambda x: "Complete_Area" if x>0 else "Incomplete_Area")(df2['Complete_Area'])

In [13]:
df2sql(df2, "complete/patch_info_update.db", patch_size, mode='replace')

In [15]:
import pickle
pickle.dump([(i%4,)+x for i,x in enumerate([("bcc",x) for x in np.array_split(df['ID'].unique(),10)]+[("complete",x) for x in np.array_split(df2['ID'].unique(),14)])],open("job_ids.pkl","wb"))

